# Installing libraries

In [1]:
!pip install google-ai-generativelanguage==0.6.15 --force-reinstall --upgrade --quiet


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.1.3 requires google-ai-generativelanguage<0.7.0,>=0.6.16, but you have google-ai-generativelanguage 0.6.15 which is incompatible.

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install google-generativeai --upgrade --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install -q youtube-transcript-api langchain-community langchain-openai faiss-cpu tiktoken python-dotenv langchain_google_genai   --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.17 which is incompatible.


In [4]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled

from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings


from langchain_community.vectorstores import FAISS

from langchain_core.prompts import PromptTemplate

from langchain_google_genai import ChatGoogleGenerativeAI

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough,RunnableLambda

from dotenv import load_dotenv
import os

load_dotenv()


True

# Step1 Indexing
a.  (Document Ingestion)

In [6]:
video_id = "LPZh9BOjkQs"  # only the ID, not full URL
try:
    # If you don't care which language, this returns the "best" one
    transcript_list = YouTubeTranscriptApi.get_transcript(video_id, languages=["en"])

    # Flatten it to plain text
    transcript = " ".join(chunk["text"] for chunk in transcript_list) # basically here we concatenating items of genrerator by space
    print(transcript)

except Exception as e:
    print("Error:" , e)

Imagine you happen across a short movie script that describes a scene between a person and their AI assistant. The script has what the person asks the AI, but the AI's response has been torn off. Suppose you also have this powerful magical machine that can take any text and provide a sensible prediction of what word comes next. You could then finish the script by feeding in what you have to the machine, seeing what it would predict to start the AI's answer, and then repeating this over and over with a growing script completing the dialogue. When you interact with a chatbot, this is exactly what's happening. A large language model is a sophisticated mathematical function that predicts what word comes next for any piece of text. Instead of predicting one word with certainty, though, what it does is assign a probability to all possible next words. To build a chatbot, you lay out some text that describes an interaction between a user and a hypothetical AI assistant, add on whatever the use

# Step1
b. (Text Splitting)

In [7]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.create_documents([transcript])

# Step 1c & 1d - Indexing (Embedding Generation and Storing in Vector Store)

In [8]:
chunks

[Document(metadata={}, page_content="Imagine you happen across a short movie script that describes a scene between a person and their AI assistant. The script has what the person asks the AI, but the AI's response has been torn off. Suppose you also have this powerful magical machine that can take any text and provide a sensible prediction of what word comes next. You could then finish the script by feeding in what you have to the machine, seeing what it would predict to start the AI's answer, and then repeating this over and over with a growing script completing the dialogue. When you interact with a chatbot, this is exactly what's happening. A large language model is a sophisticated mathematical function that predicts what word comes next for any piece of text. Instead of predicting one word with certainty, though, what it does is assign a probability to all possible next words. To build a chatbot, you lay out some text that describes an interaction between a user and a hypothetical 

In [9]:
import faiss
print(faiss.__version__)

1.10.0


In [13]:
huggingaFceApikey =  os.getenv("HUGGINGFACEHUB_API_TOKEN")
embeddings = HuggingFaceInferenceAPIEmbeddings(model_name="intfloat/e5-base-v2",api_key=huggingaFceApikey)
embeddings = HuggingFaceInferenceAPIEmbeddings(model_name="intfloat/e5-base-v2",api_key=huggingaFceApikey)
vector_store = FAISS.from_documents(chunks,embeddings)

In [14]:
vector_store.index_to_docstore_id # for see IDs of stored embeddings, and their corresponding index in the vecotor store(FAISS)

{0: '02a9af90-2664-47a8-958a-373b6952e6d4',
 1: '9a46a1f3-25e1-4919-b351-a15c6c12013e',
 2: 'fe916d89-5d37-4d10-880c-be2d8ec73cac',
 3: 'c22d97b8-4d49-4a9c-9542-49bd58abf2e8',
 4: '9b591ce1-c09d-479e-8eed-760ec5d19e10',
 5: '998f0e37-2c1b-4155-8ead-95fa374c70ba',
 6: 'f6f398fb-153e-4504-ba03-d5f749718187',
 7: '3f61038a-dafa-4949-9301-7b25092015da',
 8: 'c79889d3-0e0f-46af-9973-fa3d36976a3d',
 9: '2ac1d77d-3c9e-45bc-87c9-9a9749b38937'}

In [15]:
vector_store.get_by_ids(['e531f253-b7e6-4ed1-a2c3-79d782096073'])

[]

# Step 2 -> Retrieval

In [16]:
retriever = vector_store.as_retriever(search_type="similarity",search_kwargs={"k": 4})

In [17]:
retriever = vector_store.as_retriever(search_type="similarity",search_kwargs={"k": 4})

In [18]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceInferenceAPIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001CA70AA92B0>, search_kwargs={'k': 4})

In [19]:
question = 'What this video related to?'
context_text = retriever.invoke(question)
context_text

[Document(id='2ac1d77d-3c9e-45bc-87c9-9a9749b38937', metadata={}, page_content='the other steps in a transformer. Also, on my second channel I just posted a talk I gave a couple months ago about this topic for the company TNG in Munich. Sometimes I actually prefer the content I make as a casual talk rather than a produced video, but I leave it up to you which one of these feels like the better follow-on.'),
 Document(id='02a9af90-2664-47a8-958a-373b6952e6d4', metadata={}, page_content="Imagine you happen across a short movie script that describes a scene between a person and their AI assistant. The script has what the person asks the AI, but the AI's response has been torn off. Suppose you also have this powerful magical machine that can take any text and provide a sensible prediction of what word comes next. You could then finish the script by feeding in what you have to the machine, seeing what it would predict to start the AI's answer, and then repeating this over and over with a gr

# Step 3 -> Augmentation

In [20]:
prompt = PromptTemplate(
    template="""You are an assistant for question-answering tasks. Use the following pieces of retrieved context(retrieved from one youtube video) to answer the question. If you don't know the answer, just say that you don't know.
    \nnQuestion: {question}
    \nnRetrieved Context: {context}  """,
    input_variables=["question", "context"],
)



In [21]:
context = '\n\n'.join([doc.page_content for doc in context_text])

In [22]:
final_prompt = prompt.invoke({'context':context,'question':question})
final_prompt

StringPromptValue(text="You are an assistant for question-answering tasks. Use the following pieces of retrieved context(retrieved from one youtube video) to answer the question. If you don't know the answer, just say that you don't know.\n    \nnQuestion: What this video related to?\n    \nnRetrieved Context: the other steps in a transformer. Also, on my second channel I just posted a talk I gave a couple months ago about this topic for the company TNG in Munich. Sometimes I actually prefer the content I make as a casual talk rather than a produced video, but I leave it up to you which one of these feels like the better follow-on.\n\nImagine you happen across a short movie script that describes a scene between a person and their AI assistant. The script has what the person asks the AI, but the AI's response has been torn off. Suppose you also have this powerful magical machine that can take any text and provide a sensible prediction of what word comes next. You could then finish the s

# Generation

In [23]:
# make llm for genration


In [25]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro",api_key=os.getenv('GOOGLE_API_KEY'))

In [26]:
answer = llm.invoke(final_prompt)
answer

AIMessage(content='This video explains how large language models (LLMs) work, specifically focusing on how they are used in chatbots.  It covers the process of predicting the next word in a sequence, the architecture of the model including feed-forward neural networks, the massive scale of computation required for training (pre-training), and the subsequent reinforcement learning with human feedback to refine the model for chatbot applications.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-pro-002', 'safety_ratings': []}, id='run-f6a1d948-e5e3-4f94-989f-cc952f305335-0', usage_metadata={'input_tokens': 727, 'output_tokens': 80, 'total_tokens': 807, 'input_token_details': {'cache_read': 0}})

In [27]:
answer.content

'This video explains how large language models (LLMs) work, specifically focusing on how they are used in chatbots.  It covers the process of predicting the next word in a sequence, the architecture of the model including feed-forward neural networks, the massive scale of computation required for training (pre-training), and the subsequent reinforcement learning with human feedback to refine the model for chatbot applications.'

In [28]:
parser = StrOutputParser()

# making chains

In [29]:
def get_context(context_text):
    context = '\n\n'.join([doc.page_content for doc in context_text])
    return context

In [32]:

# Parallel chain
parallelChain = RunnableParallel({
    'context': retriever | RunnableLambda(get_context),
    'question':RunnablePassthrough()

})

chain = parallelChain | prompt | llm | parser
chain.invoke('I would like if you summarize this video')

'This video explains how large language models (LLMs) work, specifically in the context of chatbots.  LLMs are essentially sophisticated mathematical functions that predict the probability of the next word in a sequence of text.  They are trained on massive amounts of text data, learning to predict the next word by comparing their predictions to the actual next word and adjusting their internal parameters accordingly through a process called backpropagation.  The sheer scale of data and parameters involved in training these models is immense.  The video also mentions that while the framework of these models is designed by researchers, the specific behavior emerges from the training process, making it difficult to understand precisely why a model makes certain predictions.  Finally, the video creator points viewers to other resources on their channels that delve deeper into the technical details of transformers and attention mechanisms, key components of LLMs.'

In [39]:
import streamlit as st

streamlit.set_page_config(layout="wide")

def main():
    st.title("Youtube Video Summarization")
    st.sidebar.title("Input")
    video_id = st.sidebar.text_input("Video ID", "LPZh9BOjkQs")
    st.sidebar.text("Note: This tool uses YouTube Transcript API for video transcription and HuggingFace's LLM for summarization.")
    

2025-04-27 22:14:38.078 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Streamlit requires raw Python (.py) files, not .ipynb.
For more information, please see https://docs.streamlit.io
